In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.049301862716674805
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 5.26385885609599
200000 7.934640036134246
300000 8.14017004797705
400000 8.782178606701354
500000 10.89161744528523
600000 12.172712770251323
700000 12.245414574231743
800000 11.200840180299567
900000 12.197296505038002
1000000 14.438957275084903
1100000 14.106253816370517
1200000 7.977128399643036
1300000 19.427285122375658
1400000 21.94613385517788
1500000 23.596057332769014
1600000 22.334180030786158
1700000 23.64171050526595
1800000 22.638477857747542
1900000 22.416068736136843
2000000 22.261154753694715
2100000 22.67623665087923
2200000 22.31861014520611
2300000 5.807992965201781
2400000 24.979820193991888
2500000 27.923185512559353
2600000 27.460225391945766
2700000 26.77447765501987
2800000 26.794548285601575
2900000 27.679534144708096
3000000 25.79

29400000 20.97716043893462
29500000 7.771329583943546
29600000 21.417021478737677
29700000 21.28104662981161
29800000 21.222439146569677
29900000 20.26244476521668
30000000 21.269401496386337
30100000 19.372062663564126
30200000 19.904352643500495
30300000 19.099887023697676
30400000 5.762860274167002
30500000 20.277490095858695
30600000 18.358655556603196
30700000 20.335098896448116
30800000 20.079651877939057
30900000 20.455276714148052
31000000 20.858907695387494
31100000 20.13992366455906
31200000 21.448366465292768
31300000 9.110645711198389
31400000 21.62296075448429
31500000 21.134457218800907
31600000 21.37096665263202
31700000 17.906921608215676
31800000 10.914520626092404
31900000 21.299375535280596
32000000 19.93246152769835
32100000 20.37694915313947
32200000 20.068229970961333
32300000 19.396813942967434
32400000 20.487872101316896
32500000 18.253538839815665
32600000 12.429739479899526
32700000 14.054434112593576
32800000 18.701154163202087
32900000 19.894739614515096
330

59100000 17.45826246971408
59200000 18.058134949791224
59300000 17.15553858699041
59400000 16.370218142640518
59500000 17.560736676076363
59600000 7.435196404442863
59700000 15.741145387012962
59800000 17.418456056637037
59900000 16.9372367468881
60000000 17.38165819788536
60100000 17.12589009659937
60200000 19.478059964164412
60300000 8.100276556663262
60400000 19.150002896325184
60500000 19.16785071314467
60600000 18.54572301342132
60700000 19.95331024583603
60800000 18.706207397627608
60900000 20.103526345389092
61000000 18.209455590477944
61100000 19.017835385072452
61200000 8.25596285855563
61300000 16.44240776407827
61400000 18.592039276815754
61500000 16.71536141992939
61600000 17.78035233080218
61700000 17.05270651535081
61800000 14.586545417242096
61900000 8.768791147000105
62000000 16.097452858470128
62100000 17.873872886729885
62200000 16.43290523403861
62300000 17.07859688371203
62400000 17.941826552577346
62500000 9.3798550359629
62600000 17.667438333204895
62700000 17.967

88800000 9.640533852742445
88900000 10.315116291355134
89000000 14.44286787729479
89100000 14.21344063231082
89200000 14.504336227324426
89300000 6.671230138915241
89400000 14.91005340853664
89500000 14.55898863526555
89600000 15.539565637530512
89700000 15.51665334263857
89800000 15.933656777734608
89900000 16.251168752093413
90000000 16.226303011120997
90100000 15.975661443245867
90200000 16.688621495212427
90300000 16.386047786522695
90400000 15.564936731844675
90500000 14.787059399727319
90600000 12.256414983169671
90700000 8.493094520963012
90800000 14.354597627313584
90900000 14.37961038006926
91000000 15.135093983257045
91100000 4.21986134834069
91200000 15.569069852344896
91300000 14.660914340548922
91400000 14.06652405418465
91500000 8.260016781593436
91600000 12.806489394958108
91700000 14.653412731208366
91800000 15.642724808152144
91900000 15.640179854183595
92000000 16.283127387452666
92100000 15.67805918305234
92200000 16.371042350795
92300000 16.20959087557515
92400000 1

117800000 8.716971515792416
117900000 8.541602177513584
118000000 10.23770806424431
118100000 5.931217616107877
118200000 12.244997312703534
118300000 5.557245715961785
118400000 12.789618328158678
118500000 12.190071428864128
118600000 12.589491610288203
118700000 12.551164954453483
118800000 12.502863999527584
118900000 12.88790320675524
119000000 13.075886665346045
119100000 13.779624858417513
119200000 13.84040647626393
119300000 13.78652123859432
119400000 13.461449488640458
119500000 12.106998125873984
119600000 12.438831188662913
119700000 12.16605555181159
119800000 12.064420851557495
119900000 12.288458478136484
120000000 11.795296039560727
120100000 5.71974117873006
120200000 11.161593547218263
120300000 4.880475736312787
120400000 11.039973442536592
120500000 7.197794502186484
120600000 10.94293849351075
120700000 11.873017773353666
120800000 12.412724010691447
120900000 12.00064544854856
121000000 12.050110247926545
121100000 12.309820121913692
121200000 12.371806008904906


...

175500000 9.95805081462019
175600000 9.089656118834515
175700000 9.087381083872385
175800000 8.808244303552875
175900000 9.060169481954167
176000000 8.9180532406285
176100000 8.560449105010807
176200000 8.967121164823205
176300000 3.8355532280363374
176400000 8.840054881985195
176500000 3.7633968914075293
176600000 7.488112333208859
176700000 5.768858836148252
176800000 6.064376291402519
176900000 9.122382526512158
177000000 8.436882195129796
177100000 8.854865434061109
177200000 8.820828153643241
177300000 8.691838423562457
177400000 9.409034941625674
177500000 9.410048376291563
177600000 9.937531788276946
177700000 9.465092972475976
177800000 9.272740684256958
177900000 8.84276885870434
178000000 8.312289444039076
178100000 8.640751676976011
178200000 8.411923292048117
178300000 8.203018403300266
178400000 8.260292706921131
178500000 5.187358655794121
178600000 7.11707653883471
178700000 6.65483941466064
178800000 3.9956830426983174
178900000 7.86906286590904
179000000 3.616417989070

204800000 7.270608197936386
204900000 7.155579407854139
205000000 6.850718458630443
205100000 6.831831222814176
205200000 6.581026587610469
205300000 2.9254610919409894
205400000 6.74262671191995
205500000 6.329134866828809
205600000 6.5024159257605385
205700000 4.386092941659544
205800000 4.491353823069496
205900000 6.64352017145431
206000000 6.551403793526136
206100000 6.699718365107272
206200000 2.4331201873785258
206300000 6.748803046692982
206400000 6.786107586695956
206500000 6.962800095248679
206600000 7.071993921089652
206700000 7.31456607691286
206800000 7.284682887384267
206900000 7.125117272599022
207000000 7.168121900591509
207100000 7.219005878585538
207200000 6.630171574487126
207300000 6.677918941800873
207400000 6.329041335016678
207500000 4.008411722967253
207600000 5.448944032995552
207700000 6.256475377129733
207800000 6.352876885372828
207900000 6.387950707442552
208000000 1.8391277401266892
208100000 6.6860488698082206
208200000 6.143484184666114
208300000 6.306547

234000000 4.9285332266214725
234100000 4.948185213920693
234200000 5.0118070208497185
234300000 2.0090523747383853
234400000 5.02262927624157
234500000 4.489003189820138
234600000 4.4355817295467554
234700000 4.394014194597423
234800000 4.423309383973333
234900000 3.8816680656687317
235000000 2.097566407076816
235100000 4.444193695294619
235200000 4.31397954082203
235300000 4.281778781000286
235400000 4.5157487886011864
235500000 4.476102174983952
235600000 3.1341529519515037
235700000 3.4088676262338296
235800000 4.845773800838083
235900000 4.811325238827169
236000000 4.687779594924274
236100000 5.136372952858727
236200000 4.664383532089869
236300000 4.850072578294151
236400000 4.771771448294634
236500000 2.95033219558531
236600000 3.842214064674109
236700000 4.367095893359045
236800000 4.444240863162061
236900000 4.1464031600001485
237000000 4.166156431477
237100000 4.20766319913143
237200000 1.8392552862331037
237300000 3.910598657190545
237400000 4.114928509197682
237500000 4.23586

262900000 2.28663809541297
263000000 2.7489463648132215
263100000 2.6004109468208587
263200000 2.6569292519696748
263300000 1.083843060141845
263400000 2.822611120162616
263500000 2.5529390077006817
263600000 2.5456226596102614
263700000 2.630055606050564
263800000 2.331879605305662
263900000 2.5498205530193085
264000000 2.3022792969515455
264100000 2.434502136353195
264200000 0.7019122555074924
264300000 2.416993777812173
264400000 2.4125519054837823
264500000 2.378199020842936
264600000 2.6223520897289014
264700000 2.4900973300382
264800000 2.536372898487217
264900000 2.569948427012334
265000000 2.5075685803678334
265100000 0.914737213903523
265200000 2.4537072908010815
265300000 2.621903697702885
265400000 2.483417218136317
265500000 1.5457297546552287
265600000 2.0512109805014433
265700000 2.376278056185666
265800000 2.573571800495277
265900000 2.314230262671669
266000000 2.417899629874733
266100000 2.3604204397481645
266200000 2.264316771221757
266300000 2.285451010300063
26640000

291400000 0.4551911526322961
291500000 0.4401162348670794
291600000 0.4291728398778969
291700000 0.45495164360493423
291800000 0.43447185274073485
291900000 0.4290011655130353
292000000 0.421719360554268
292100000 0.4213339587920772
292200000 0.3612999680943986
292300000 0.08848959962582588
292400000 0.32380247496097286
292500000 0.2980761038248406
292600000 0.29069839200870196
292700000 0.2765154177918666
292800000 0.2799632343129549
292900000 0.2775279574928085
293000000 0.2591286355700725
293100000 0.25655501141657433
293200000 0.23672608271074297
293300000 0.24056242982058723
293400000 0.09919640171803037
293500000 0.19358618562832475
293600000 0.22171994864664804
293700000 0.1958690832043522
293800000 0.19902816595211625
293900000 0.19827672781589295
294000000 0.2031971444836855
294100000 0.17756697782662512
294200000 0.157929666852004
294300000 0.15169583798962172
294400000 0.15846714757098754
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.0927